In [1]:
import pandas as pd
import numpy
from pytrends.request import TrendReq
import pytrends
from statistics import mean
import time

pytrend = TrendReq()

In [2]:
states = {'AK': 'Alaska','AL': 'Alabama','AR': 'Arkansas','AZ': 'Arizona','CA': 'California','CO': 'Colorado','CT': 'Connecticut','DE': 'Delaware','FL': 'Florida','GA': 'Georgia','HI': 'Hawaii','IA': 'Iowa','ID': 'Idaho','IL': 'Illinois','IN': 'Indiana','KS': 'Kansas','KY': 'Kentucky','LA': 'Louisiana','MA': 'Massachusetts','MD': 'Maryland','ME': 'Maine','MI': 'Michigan','MN': 'Minnesota','MO': 'Missouri','MS': 'Mississippi','MT': 'Montana','NC': 'North Carolina','ND': 'North Dakota','NE': 'Nebraska','NH': 'New Hampshire','NJ': 'New Jersey','NM': 'New Mexico','NV': 'Nevada','NY': 'New York','OH': 'Ohio','OK': 'Oklahoma','OR': 'Oregon','PA': 'Pennsylvania','RI': 'Rhode Island','SC': 'South Carolina','SD': 'South Dakota','TN': 'Tennessee','TX': 'Texas','UT': 'Utah','VA': 'Virginia','VT': 'Vermont','WA': 'Washington','WI': 'Wisconsin','WV': 'West Virginia',    'WY': 'Wyoming'}
stopwords = [' & Company', 'Holdings', 'Automobiles', ' Inc.', 'Communications', 'Corporation', 'Brands', 'The', 'Walt', 'Technologies', '& Co.', 'Co.', '(CVS Health)', 'Labs', ' Burger', 'International', 'Group', '\'' ]

In [4]:
def getCorps(): #get corps and create empty dataframe
    
    corpsDF = pd.read_csv('corps.csv')
    corpsRaw = corpsDF['companies'].tolist()
    corps = []
    for corp in corpsRaw:
        for word in stopwords:
            if word in corp:
                corp = corp.replace(word,'')
                print("replaced: \"{}\"  in  \"{}\"".format(word,corp))
        corps.append(str(corp).strip())
    return sorted(corps)

corps = getCorps()
print(len(corps))

trendsDF = pd.DataFrame(columns=[sorted(corps)], index=[states.values()][0])
trendsDF.head()

replaced: "'"  in  "Dicks Sporting Goods"
replaced: "'"  in  "Dominos Pizza"
replaced: "Corporation"  in  "FedEx "
replaced: "Corporation"  in  "Fox "
replaced: " Inc."  in  "Grubhub"
replaced: "International"  in  "Honeywell "
replaced: "Technologies"  in  "Huawei "
replaced: " Burger"  in  "In-n-Out"
replaced: "'"  in  "Kohls"
replaced: "'"  in  "Lowes"
replaced: "'"  in  "Macys"
replaced: "'"  in  "McDonalds"
replaced: "'"  in  "Papa Johns"
replaced: "The"  in  " Clorox Company"
replaced: "The"  in  " Hershey Company"
replaced: "The"  in  " Home Depot"
replaced: "Group"  in  "UnitedHealth "
replaced: "Walt"  in  " Disney"
replaced: "'"  in  "Wendys"
replaced: "Brands"  in  "Yum! "
191


,3M Company,AT&T,Absolut,Adidas,Airbnb,Aldi,Allstate,Amazon,American Airlines,American Express,...,Wegmans,Wells Fargo,Wendys,White Claw,Whole Foods,Wish.com,Woodford Reserve,Yum!,Zoom,eBay
Alaska,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Alabama,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Arkansas,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Arizona,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
California,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
def getQuery(tF): # retrieve and format query into dataframe trendsDF
    
    trendsDF = pd.DataFrame(columns=[sorted(corps)], index=[states.values()][0])

    TF = '{}-01-01 {}-12-31'.format(tF, tF) #format timeframe for query based on year passed. Can be customized down to daily or weekly queries
    print("Searching within timeframe:", TF)
    
    for corp in sorted(corps):
        
        print('Running Google trends query for:', corp, tF)
                
        try: # normal, no errors
            pytrend.build_payload(kw_list=[corp], timeframe = TF, geo = 'US') #searchState.replace(' ', 'US-') ---- Build this payload before search with queries
        except: # google has sent an html error (usually rate limit or syntax)
            print("Google Error")
            time.sleep(60)
            pytrend.build_payload(kw_list=[corp], timeframe = TF, geo = 'US')

        df = pytrend.interest_by_region(resolution = "country", inc_low_vol = True, inc_geo_code = False).reset_index() # establish dataframe to be manipulated
        
        for i in df.itertuples(index = False): # iterate through each row of the intake data

            print(i[0], 'score: ', i[1])
            trendsDF.at[i[0],corp] = i[1]

    filename = "corp_trends{}{}".format(tF,'.csv')
    trendsDF.to_csv(filename)
    print("\n======== Query finished. Data copied to:", filename, " ========\n")

for y in [2004, 2006, 2008, 2010, 2012, 2016, 2018, 2020]:
    getQuery(y)
    time.sleep(60)

Searching within timeframe: 2004-01-01 2004-12-31
Running Google trends query for: 3M Company 2004
Alabama score:  43
Alaska score:  0
Arizona score:  19
Arkansas score:  0
California score:  24
Colorado score:  18
Connecticut score:  42
Delaware score:  0
District of Columbia score:  0
Florida score:  19
Georgia score:  36
Hawaii score:  38
Idaho score:  52
Illinois score:  30
Indiana score:  29
Iowa score:  23
Kansas score:  23
Kentucky score:  29
Louisiana score:  0
Maine score:  0
Maryland score:  27
Massachusetts score:  9
Michigan score:  20
Minnesota score:  100
Mississippi score:  0
Missouri score:  37
Montana score:  0
Nebraska score:  0
Nevada score:  34
New Hampshire score:  0
New Jersey score:  29
New Mexico score:  46
New York score:  38
North Carolina score:  45
North Dakota score:  0
Ohio score:  33
Oklahoma score:  24
Oregon score:  13
Pennsylvania score:  32
Rhode Island score:  0
South Carolina score:  22
South Dakota score:  0
Tennessee score:  13
Texas score:  21
Ut

Alabama score:  33
Alaska score:  61
Arizona score:  68
Arkansas score:  61
California score:  77
Colorado score:  53
Connecticut score:  59
Delaware score:  38
District of Columbia score:  62
Florida score:  60
Georgia score:  42
Hawaii score:  71
Idaho score:  25
Illinois score:  67
Indiana score:  41
Iowa score:  43
Kansas score:  52
Kentucky score:  31
Louisiana score:  48
Maine score:  46
Maryland score:  49
Massachusetts score:  65
Michigan score:  33
Minnesota score:  35
Mississippi score:  29
Missouri score:  79
Montana score:  18
Nebraska score:  52
Nevada score:  58
New Hampshire score:  53
New Jersey score:  43
New Mexico score:  77
New York score:  60
North Carolina score:  46
North Dakota score:  10
Ohio score:  35
Oklahoma score:  80
Oregon score:  47
Pennsylvania score:  29
Rhode Island score:  52
South Carolina score:  26
South Dakota score:  23
Tennessee score:  47
Texas score:  100
Utah score:  39
Vermont score:  34
Virginia score:  48
Washington score:  49
West Virgi

Alabama score:  62
Alaska score:  38
Arizona score:  52
Arkansas score:  47
California score:  54
Colorado score:  67
Connecticut score:  46
Delaware score:  85
District of Columbia score:  48
Florida score:  50
Georgia score:  51
Hawaii score:  54
Idaho score:  92
Illinois score:  51
Indiana score:  54
Iowa score:  50
Kansas score:  49
Kentucky score:  59
Louisiana score:  55
Maine score:  36
Maryland score:  54
Massachusetts score:  58
Michigan score:  45
Minnesota score:  52
Mississippi score:  59
Missouri score:  56
Montana score:  49
Nebraska score:  43
Nevada score:  72
New Hampshire score:  76
New Jersey score:  53
New Mexico score:  49
New York score:  56
North Carolina score:  78
North Dakota score:  76
Ohio score:  49
Oklahoma score:  35
Oregon score:  76
Pennsylvania score:  50
Rhode Island score:  60
South Carolina score:  31
South Dakota score:  35
Tennessee score:  100
Texas score:  52
Utah score:  38
Vermont score:  72
Virginia score:  52
Washington score:  73
West Virgi

Alabama score:  25
Alaska score:  0
Arizona score:  13
Arkansas score:  17
California score:  14
Colorado score:  12
Connecticut score:  18
Delaware score:  15
District of Columbia score:  18
Florida score:  24
Georgia score:  13
Hawaii score:  7
Idaho score:  19
Illinois score:  30
Indiana score:  19
Iowa score:  40
Kansas score:  17
Kentucky score:  0
Louisiana score:  13
Maine score:  7
Maryland score:  16
Massachusetts score:  34
Michigan score:  32
Minnesota score:  29
Mississippi score:  10
Missouri score:  14
Montana score:  16
Nebraska score:  29
Nevada score:  13
New Hampshire score:  13
New Jersey score:  23
New Mexico score:  9
New York score:  15
North Carolina score:  9
North Dakota score:  100
Ohio score:  14
Oklahoma score:  14
Oregon score:  14
Pennsylvania score:  35
Rhode Island score:  59
South Carolina score:  25
South Dakota score:  92
Tennessee score:  25
Texas score:  7
Utah score:  0
Vermont score:  46
Virginia score:  21
Washington score:  4
West Virginia score

Alabama score:  48
Alaska score:  26
Arizona score:  26
Arkansas score:  68
California score:  17
Colorado score:  28
Connecticut score:  22
Delaware score:  21
District of Columbia score:  25
Florida score:  28
Georgia score:  100
Hawaii score:  10
Idaho score:  21
Illinois score:  33
Indiana score:  37
Iowa score:  34
Kansas score:  29
Kentucky score:  52
Louisiana score:  38
Maine score:  20
Maryland score:  20
Massachusetts score:  17
Michigan score:  20
Minnesota score:  32
Mississippi score:  15
Missouri score:  27
Montana score:  28
Nebraska score:  53
Nevada score:  13
New Hampshire score:  21
New Jersey score:  12
New Mexico score:  21
New York score:  25
North Carolina score:  28
North Dakota score:  35
Ohio score:  29
Oklahoma score:  47
Oregon score:  17
Pennsylvania score:  27
Rhode Island score:  31
South Carolina score:  31
South Dakota score:  58
Tennessee score:  38
Texas score:  30
Utah score:  32
Vermont score:  8
Virginia score:  28
Washington score:  20
West Virgin

...

Alabama score:  25
Alaska score:  7
Arizona score:  94
Arkansas score:  23
California score:  25
Colorado score:  56
Connecticut score:  14
Delaware score:  27
District of Columbia score:  22
Florida score:  30
Georgia score:  19
Hawaii score:  48
Idaho score:  35
Illinois score:  36
Indiana score:  45
Iowa score:  18
Kansas score:  59
Kentucky score:  16
Louisiana score:  33
Maine score:  18
Maryland score:  29
Massachusetts score:  19
Michigan score:  21
Minnesota score:  21
Mississippi score:  17
Missouri score:  76
Montana score:  7
Nebraska score:  54
Nevada score:  100
New Hampshire score:  34
New Jersey score:  10
New Mexico score:  83
New York score:  16
North Carolina score:  20
North Dakota score:  5
Ohio score:  30
Oklahoma score:  53
Oregon score:  34
Pennsylvania score:  23
Rhode Island score:  22
South Carolina score:  12
South Dakota score:  8
Tennessee score:  29
Texas score:  65
Utah score:  15
Vermont score:  13
Virginia score:  15
Washington score:  30
West Virginia 

Alabama score:  45
Alaska score:  10
Arizona score:  63
Arkansas score:  34
California score:  72
Colorado score:  52
Connecticut score:  37
Delaware score:  51
District of Columbia score:  65
Florida score:  81
Georgia score:  58
Hawaii score:  57
Idaho score:  47
Illinois score:  60
Indiana score:  34
Iowa score:  22
Kansas score:  65
Kentucky score:  38
Louisiana score:  38
Maine score:  19
Maryland score:  58
Massachusetts score:  45
Michigan score:  51
Minnesota score:  48
Mississippi score:  32
Missouri score:  45
Montana score:  24
Nebraska score:  17
Nevada score:  67
New Hampshire score:  27
New Jersey score:  58
New Mexico score:  95
New York score:  58
North Carolina score:  39
North Dakota score:  18
Ohio score:  43
Oklahoma score:  49
Oregon score:  42
Pennsylvania score:  54
Rhode Island score:  62
South Carolina score:  44
South Dakota score:  11
Tennessee score:  44
Texas score:  69
Utah score:  53
Vermont score:  11
Virginia score:  48
Washington score:  100
West Virgi

Alabama score:  22
Alaska score:  34
Arizona score:  32
Arkansas score:  26
California score:  30
Colorado score:  29
Connecticut score:  34
Delaware score:  31
District of Columbia score:  100
Florida score:  33
Georgia score:  27
Hawaii score:  23
Idaho score:  32
Illinois score:  32
Indiana score:  27
Iowa score:  37
Kansas score:  32
Kentucky score:  25
Louisiana score:  21
Maine score:  36
Maryland score:  36
Massachusetts score:  33
Michigan score:  35
Minnesota score:  30
Mississippi score:  18
Missouri score:  33
Montana score:  26
Nebraska score:  24
Nevada score:  33
New Hampshire score:  33
New Jersey score:  39
New Mexico score:  29
New York score:  36
North Carolina score:  26
North Dakota score:  10
Ohio score:  31
Oklahoma score:  29
Oregon score:  36
Pennsylvania score:  31
Rhode Island score:  34
South Carolina score:  29
South Dakota score:  19
Tennessee score:  23
Texas score:  23
Utah score:  20
Vermont score:  15
Virginia score:  39
Washington score:  38
West Virgi

Alabama score:  9
Alaska score:  0
Arizona score:  19
Arkansas score:  9
California score:  11
Colorado score:  13
Connecticut score:  27
Delaware score:  14
District of Columbia score:  25
Florida score:  17
Georgia score:  14
Hawaii score:  15
Idaho score:  6
Illinois score:  14
Indiana score:  13
Iowa score:  15
Kansas score:  17
Kentucky score:  10
Louisiana score:  10
Maine score:  6
Maryland score:  15
Massachusetts score:  14
Michigan score:  8
Minnesota score:  100
Mississippi score:  10
Missouri score:  19
Montana score:  0
Nebraska score:  13
Nevada score:  28
New Hampshire score:  12
New Jersey score:  16
New Mexico score:  8
New York score:  15
North Carolina score:  15
North Dakota score:  7
Ohio score:  11
Oklahoma score:  10
Oregon score:  7
Pennsylvania score:  16
Rhode Island score:  24
South Carolina score:  12
South Dakota score:  0
Tennessee score:  16
Texas score:  17
Utah score:  10
Vermont score:  0
Virginia score:  16
Washington score:  9
West Virginia score:  1

Alabama score:  50
Alaska score:  39
Arizona score:  43
Arkansas score:  46
California score:  28
Colorado score:  57
Connecticut score:  45
Delaware score:  35
District of Columbia score:  19
Florida score:  56
Georgia score:  69
Hawaii score:  23
Idaho score:  35
Illinois score:  54
Indiana score:  75
Iowa score:  36
Kansas score:  52
Kentucky score:  80
Louisiana score:  59
Maine score:  28
Maryland score:  42
Massachusetts score:  43
Michigan score:  66
Minnesota score:  32
Mississippi score:  78
Missouri score:  48
Montana score:  45
Nebraska score:  31
Nevada score:  31
New Hampshire score:  35
New Jersey score:  47
New Mexico score:  34
New York score:  39
North Carolina score:  59
North Dakota score:  28
Ohio score:  100
Oklahoma score:  28
Oregon score:  30
Pennsylvania score:  61
Rhode Island score:  43
South Carolina score:  57
South Dakota score:  23
Tennessee score:  62
Texas score:  35
Utah score:  63
Vermont score:  17
Virginia score:  54
Washington score:  38
West Virgi

In [1]:
newTrendsDF = pd.read_csv('corp_trends2010.csv', index_col=[0])
#newTrendsDF.reset_index().set_index([0])
newTrendsDF.head()
#trendsDF.to_csv("corp_trends{}{}".format('2016','.csv'))


NameError: name 'pd' is not defined

In [86]:
newTrendsDF.head(50)

,3M Company,AT&T,Adidas,Airbnb,Aldi,Allstate,Amazon,American Airlines,American Express,Apple,...,Walgreens,Walmart,Wayfair,Wegmans,Wells Fargo,Wendy's,Wish,Yum!,Zoom Video,eBay
Alaska,0,63,28,63,1,65,82,29,32,69,...,31,37,21,0,84,39,48,0,66,50
Alabama,34,74,60,61,44,93,78,34,51,65,...,58,82,59,1,61,50,57,0,32,65
Arkansas,28,82,49,56,21,47,73,45,35,64,...,65,100,51,0,11,46,60,0,40,65
Arizona,28,28,59,80,7,56,85,100,58,70,...,74,56,53,1,86,43,58,0,48,52
California,31,48,85,88,17,42,82,42,58,88,...,34,37,53,1,61,28,55,0,100,52
Colorado,27,30,51,78,3,56,80,47,45,68,...,62,45,55,1,65,57,49,0,60,49
Connecticut,43,33,67,62,41,48,83,42,75,84,...,47,43,75,5,28,45,50,0,63,56
Delaware,0,30,52,44,36,47,100,55,45,75,...,97,54,75,18,50,35,43,0,72,57
Florida,33,48,66,85,57,56,90,87,87,72,...,71,59,65,1,69,56,54,0,51,55
Georgia,30,68,77,79,46,73,77,33,74,72,...,46,66,62,1,70,69,57,0,41,51


In [88]:
newTrendsDF[['Apple', 'Papa John\'s', 'Chick-fil-A']].T

,Alaska,Alabama,Arkansas,Arizona,California,Colorado,Connecticut,Delaware,Florida,Georgia,...,Tennessee,Texas,Utah,Virginia,Vermont,Washington,Wisconsin,West Virginia,Wyoming,District of Columbia
Apple,69,65,64,70,88,68,84,75,72,72,...,70,71,77,83,77,75,74,69,55,82
Papa John's,30,59,32,28,11,24,4,28,45,65,...,69,30,24,46,1,16,17,58,23,24
Chick-fil-A,4,59,41,36,32,40,22,52,65,100,...,59,66,39,66,8,23,24,47,16,28


In [56]:
newTrendsDF['Apple']

0      69
1      65
2      64
3      70
4      88
5      68
6      84
7      75
8      72
9      72
10     88
11     66
12     64
13     77
14     65
15     68
16     63
17     65
18     87
19     75
20     75
21     74
22    100
23     67
24     67
25     60
26     71
27     61
28     66
29     80
30     84
31     64
32     76
33     80
34     68
35     63
36     70
37     74
38     75
39     67
40     57
41     70
42     71
43     77
44     83
45     77
46     75
47     74
48     69
49     55
50     82
Name: Apple, dtype: int64

In [33]:
#Build payload before any searches:
pytrend.build_payload(kw_list=['Apple'])

#Related Queries
related_queries = pytrend.related_queries()
related_queries

#Interest over Time:
#pytrend.interest_over_time

# Get Google Keyword Suggestions
#pytrend.related_topics()

# Current trends
#pytrend.trending_searches(pn='united_states')


{'Apple': {'top':                   query  value
  0           apple watch    100
  1          apple iphone     64
  2                iphone     64
  3           apple store     47
  4              apple tv     33
  5              id apple     31
  6           apple cider     24
  7            ipad apple     20
  8                  ipad     20
  9           apple music     19
  10  apple cider vinegar     18
  11                  app     15
  12                  bmw     15
  13                 benz     14
  14                loans     14
  15            apple pay     14
  16          apple phone     14
  17          apple stock     13
  18              macbook     13
  19        apple macbook     13
  20              binance     12
  21            apple mac     12
  22                  mac     12
  23                  btc     12
  24        apple watch 3     12, 'rising':                    query   value
  0                binance  237100
  1            job hunting  173300
  2         

In [117]:
trends20 = pd.read_csv('corp_trends2020.csv', index_col=[0])
trends18 = pd.read_csv('corp_trends2018.csv', index_col=[0])
trends16 = pd.read_csv('corp_trends2016.csv', index_col=[0])

trendsD16_18 = pd.DataFrame(columns=[sorted(corps)], index=[states.values()][0])
trendsD18_20 = pd.DataFrame(columns=[sorted(corps)], index=[states.values()][0])


In [32]:


for i in states.keys():
    print(i)


AK
AL
AR
AZ
CA
CO
CT
DE
FL
GA
HI
IA
ID
IL
IN
KS
KY
LA
MA
MD
ME
MI
MN
MO
MS
MT
NC
ND
NE
NH
NJ
NM
NV
NY
OH
OK
OR
PA
RI
SC
SD
TN
TX
UT
VA
VT
WA
WI
WV
WY
